In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import time

# Load MoveNet Thunder model
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# Helper to preprocess input image
def preprocess(img):
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 256, 256)
    return tf.cast(img, dtype=tf.int32)

# Get keypoints from MoveNet
def get_keypoints(image):
    input_image = preprocess(image)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0, :, :]
    return keypoints

# Helper to calculate angle
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180 else 360 - angle

# Smoother for stability
class LandmarkSmoother:
    def __init__(self, alpha=0.6):
        self.alpha = alpha
        self.prev = None

    def smooth(self, landmark):
        if self.prev is None:
            self.prev = landmark
        else:
            self.prev = [self.alpha * p + (1 - self.alpha) * l for p, l in zip(self.prev, landmark)]
        return self.prev

# Initialize smoothing
smoothers = {i: LandmarkSmoother(0.7) for i in range(17)}

def smooth(i, coord):
    return smoothers[i].smooth(coord)

# Draw keypoints
def draw_keypoints(frame, keypoints, threshold=0.3):
    h, w, _ = frame.shape
    for idx, kp in enumerate(keypoints):
        y, x, c = kp
        if c > threshold:
            cv2.circle(frame, (int(x * w), int(y * h)), 5, (0, 255, 0), -1)

# Indices for MoveNet
LEFT_SHOULDER, RIGHT_SHOULDER = 5, 6
LEFT_ELBOW, RIGHT_ELBOW = 7, 8
LEFT_WRIST, RIGHT_WRIST = 9, 10
LEFT_HIP, RIGHT_HIP = 11, 12

# Setup webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# State tracking
counter = 0
stage = None
rep_times = []
start_time = None
prev_elbows = {'left': None, 'right': None}
FONT = cv2.FONT_HERSHEY_SIMPLEX
WHITE, RED, GREEN, BLUE, BLACK = (255, 255, 255), (0, 0, 255), (0, 255, 0), (245, 117, 16), (0, 0, 0)
BOX_ALPHA = 0.7

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    keypoints = get_keypoints(img_rgb)

    try:
        coords = {i: smooth(i, [keypoints[i][1], keypoints[i][0]]) for i in range(17)}

        # Extract required coordinates
        left_shoulder = coords[LEFT_SHOULDER]
        right_shoulder = coords[RIGHT_SHOULDER]
        left_elbow = coords[LEFT_ELBOW]
        right_elbow = coords[RIGHT_ELBOW]
        left_wrist = coords[LEFT_WRIST]
        right_wrist = coords[RIGHT_WRIST]
        left_hip = coords[LEFT_HIP]
        right_hip = coords[RIGHT_HIP]

        # Angles
        left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

        feedback_text = ""
        form_score = 100

        # Elbow movement penalty
        elbow_penalty = 0
        for side, elbow in zip(['left', 'right'], [left_elbow, right_elbow]):
            if prev_elbows[side] is not None:
                movement = np.linalg.norm(np.array(elbow) - np.array(prev_elbows[side]))
                if movement > 0.05:
                    feedback_text += f"Fix {side} elbow! "
                    elbow_penalty += 10
            prev_elbows[side] = elbow

        # Arm sync check
        sync_penalty = 10 if abs(left_angle - right_angle) > 15 else 0
        if sync_penalty: feedback_text += "Sync arms! "

        # Range check
        range_penalty = 0
        if max(left_angle, right_angle) > 170:
            feedback_text += "Curl higher! "
            range_penalty += 5
        if min(left_angle, right_angle) < 20:
            feedback_text += "Lower fully! "
            range_penalty += 5

        # Shoulder height
        if abs(left_shoulder[1] - left_hip[1]) > 0.1 or abs(right_shoulder[1] - right_hip[1]) > 0.1:
            feedback_text += "Keep shoulders down! "
            form_score -= 5

        # Wrist alignment
        if abs(left_wrist[0] - left_shoulder[0]) > 0.1 or abs(right_wrist[0] - right_shoulder[0]) > 0.1:
            feedback_text += "Align wrists under shoulders! "
            form_score -= 5

        # Torso sway
        if abs(left_shoulder[0] - left_hip[0]) > 0.1 or abs(right_shoulder[0] - right_hip[0]) > 0.1:
            feedback_text += "Don't sway torso! "
            form_score -= 5

        form_score = max(0, form_score - elbow_penalty - sync_penalty - range_penalty)
        rep_quality = "Good" if form_score >= 90 else "Bad"

        # Rep counting
        current_time = time.time()
        if left_angle > 160 and right_angle > 160:
            if stage != "down":
                start_time = current_time
            stage = "down"
        if left_angle < 30 and right_angle < 30 and stage == "down":
            stage = "up"
            counter += 1
            rep_time = current_time - start_time if start_time else 0
            rep_times.append(rep_time)
            if rep_time < 1.0:
                feedback_text += "Slow down! "

        # UI Stats Box
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (300, 180), BLUE, -1)
        cv2.addWeighted(overlay, BOX_ALPHA, frame, 1 - BOX_ALPHA, 0, frame)
        cv2.putText(frame, "Dual Bicep Curl", (10, 30), FONT, 0.7, BLACK, 2)
        cv2.putText(frame, f"Reps: {counter}", (10, 60), FONT, 0.8, WHITE, 2)
        cv2.putText(frame, f"Stage: {stage or 'N/A'}", (10, 90), FONT, 0.8, WHITE, 2)
        cv2.putText(frame, f"Form: {form_score}%", (10, 120), FONT, 0.8, WHITE, 2)
        cv2.putText(frame, f"Speed: {np.mean(rep_times):.2f}s", (10, 150), FONT, 0.8, WHITE, 2)
        cv2.putText(frame, f"Rep: {rep_quality}", (10, 180), FONT, 0.8, GREEN if rep_quality == "Good" else RED, 2)

        # Feedback Box
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 400), (640, 480), BLACK, -1)
        cv2.addWeighted(overlay, BOX_ALPHA, frame, 1 - BOX_ALPHA, 0, frame)
        cv2.putText(frame, feedback_text, (10, 450), FONT, 0.7, RED if feedback_text else GREEN, 2)

        # Draw skeleton
        draw_keypoints(frame, keypoints)

    except Exception as e:
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 400), (640, 480), BLACK, -1)
        cv2.addWeighted(overlay, BOX_ALPHA, frame, 1 - BOX_ALPHA, 0, frame)
        cv2.putText(frame, "No person detected", (10, 450), FONT, 0.7, RED, 2)

    cv2.imshow('MoveNet Dual Bicep Tracker', frame)
    if cv2.waitKey(10) & 0xFF in [ord('q'), 27]:
        break

cap.release()
cv2.destroyAllWindows()

print("\nWorkout Summary:")
print(f"Total Reps: {counter}")
print(f"Average Rep Time: {np.mean(rep_times):.2f}s" if rep_times else "No reps detected")
print(f"Final Form Score: {form_score}%")
